In [30]:
import valve.source.master_server

# get list of servers, "ip:port"
def get_servers(gamedir):
    servers = []
    with valve.source.master_server.MasterServerQuerier() as msq:
        try:
            for address in msq.find(gamedir=gamedir):
                servers = servers + [str(address[0]) + str(":") + str(address[1])]
        except:
            pass
    return(servers)
# print(get_servers("NeotokyoSource"))

['216.52.143.231:27015', '216.52.143.114:27018', '216.52.143.114:27021', '88.150.176.83:26300', '198.204.255.67:27016', '75.67.231.242:27023']


In [61]:
import valve.source.a2s

# get info() of single server
def server_status(server):
    try:
        status = dict(
                valve.source.a2s.ServerQuerier((str(server.split(":")[0]), int(server.split(":")[1]))).info()
        )
    except:
        status = "Timed out waiting for response"
        
    return(status)
# print(server_status('88.150.176.83:26300'))

{'response_type': 73,
 'protocol': 17,
 'server_name': 'Datasteal.com #2 <<( Map Playtesting )>>',
 'map': 'nt_shibuya_ctg',
 'folder': 'NeotokyoSource',
 'game': 'Neotokyo',
 'app_id': 0,
 'player_count': 0,
 'max_players': 25,
 'bot_count': 1,
 'server_type': <ServerType 100 'Dedicated'>,
 'platform': <Platform 119 'Windows'>,
 'password_protected': 0,
 'vac_enabled': 1,
 'version': '1.0.0.1'}

In [88]:
servers = get_servers("NeotokyoSource")

status = {}

for server in servers:
    status[server] = server_status(server)
    
# keeps only relevant var, drops timed out servers
status = {k:{l:m for l,m in v.items() if l in 
    ['server_name', 'map', 'player_count', 'max_players', 'password_protected']} for k,v in status.items() 
 if v != 'Timed out waiting for response'}

# sorts by player count
status = dict(sorted(status.items(), key=lambda x: x[1]['player_count'], reverse = True))

In [18]:
import valve.source.a2s
import valve.source.master_server

status = {}

with valve.source.master_server.MasterServerQuerier() as msq:
    try:
        for address in msq.find(gamedir="NeotokyoSource"):
            with valve.source.a2s.ServerQuerier(address) as server:
                info = server.info()
                players = server.players()

                status[str(address[0]) + str(":") + str(address[1])] = dict(info)
    except valve.source.NoResponseError:
        pass


{'216.52.143.231:27015': {'response_type': 73,
  'protocol': 17,
  'server_name': 'Neotokyo!',
  'map': 'nt_dawn_ctg',
  'folder': 'neotokyosource',
  'game': 'Neotokyo',
  'app_id': 0,
  'player_count': 0,
  'max_players': 32,
  'bot_count': 0,
  'server_type': <ServerType 100 'Dedicated'>,
  'platform': <Platform 119 'Windows'>,
  'password_protected': 1,
  'vac_enabled': 1,
  'version': '1.0.0.1'},
 '216.52.143.114:27018': {'response_type': 73,
  'protocol': 17,
  'server_name': 'Datasteal.com <<( HLstatsX | All Plugins )>>',
  'map': 'nt_disengage_ctg',
  'folder': 'NeotokyoSource',
  'game': 'Neotokyo',
  'app_id': 0,
  'player_count': 0,
  'max_players': 25,
  'bot_count': 1,
  'server_type': <ServerType 100 'Dedicated'>,
  'platform': <Platform 119 'Windows'>,
  'password_protected': 0,
  'vac_enabled': 1,
  'version': '1.0.0.1'},
 '216.52.143.114:27021': {'response_type': 73,
  'protocol': 17,
  'server_name': 'Datasteal.com #2 <<( Map Playtesting )>>',
  'map': 'nt_shibuya_ctg

In [73]:
{k:{l:m for l,m in v.items() if l in 
    ['server_name', 'map', 'player_count', 'max_players', 'password_protected']} for k,v in status.items() 
 if v != 'Timed out waiting for response'}

{'216.52.143.231:27015': {'server_name': 'Neotokyo!',
  'map': 'nt_dawn_ctg',
  'player_count': 0,
  'max_players': 32,
  'password_protected': 1},
 '216.52.143.114:27018': {'server_name': 'Datasteal.com <<( HLstatsX | All Plugins )>>',
  'map': 'nt_disengage_ctg',
  'player_count': 0,
  'max_players': 25,
  'password_protected': 0},
 '216.52.143.114:27021': {'server_name': 'Datasteal.com #2 <<( Map Playtesting )>>',
  'map': 'nt_shibuya_ctg',
  'player_count': 0,
  'max_players': 25,
  'password_protected': 0},
 '88.150.176.83:26300': {'server_name': 'Renraku',
  'map': 'nt_isolation_ctg',
  'player_count': 0,
  'max_players': 31,
  'password_protected': 0},
 '198.204.255.67:27016': {'server_name': 'TheFeckless Dreaming of Tokyo Nights | HLStatsX:CE | Sourcemod',
  'map': 'nt_pissalley_ctg',
  'player_count': 0,
  'max_players': 24,
  'password_protected': 0}}

In [19]:
address

('198.204.255.67', 27016)

In [46]:
{k:v for k,v in status.items() if v['password_protected'] == 1} != {}

True

In [57]:
# All Servers Status
embed = discord.Embed(# title="Title", 
                      description="```css\nAll Servers```", 
                      color=0xF6B26B)
for ii in status.keys():
    embed.add_field(name=status[ii]['server_name'], 
                    value=str(status[ii]['player_count']) + "/" + str(status[ii]['max_players']) + " :: " + str(status[ii]['map']), 
                    inline=False)

# Active Servers
embed = discord.Embed(# title="Title", 
                      description="```css\nActive Servers```", 
                      color=0x00FF00)
if {k:v for k,v in status.items() if v['player_count'] > 0} != {}:
    for ii in {k:v for k,v in status.items() if v['password_protected'] == 1}.keys():
            embed.add_field(name=status[ii]['server_name'], 
                    value=str(status[ii]['player_count']) + "/" + str(status[ii]['max_players']) + " :: " + str(status[ii]['map']), 
                    inline=False)
else:
    embed.add_field(name="No one is playing",
                    value=":(",
                    inline=False)
    
            
embed.to_dict()

{'fields': [{'inline': False, 'name': 'No one is playing', 'value': ':('}],
 'color': 65280,
 'type': 'rich',
 'description': '```css\nActive Servers```'}

In [1]:
import discord
import valve.source.a2s
import valve.source.master_server

client = discord.Client()

@client.event
async def on_message(message):
# outputs status of all nt servers
    if message.content.startswith('!all'):

# queries something master server 
# creates dict status, where key is "server ip:port"
        status = {}
        with valve.source.master_server.MasterServerQuerier() as msq:
            try:
                for address in msq.find(gamedir="NeotokyoSource"):
                    with valve.source.a2s.ServerQuerier(address) as server:
                        info = server.info()
                        players = server.players()
                        status[str(address[0]) + str(":") + str(address[1])] = dict(info)
            except valve.source.NoResponseError:
                pass

        embed = discord.Embed(description="```css\nAll Servers```", 
                      color=0xF6B26B)
    
        for ii in status.keys():
            embed.add_field(name=status[ii]['server_name'], 
                            value=str(status[ii]['player_count']) + "/" + str(status[ii]['max_players']) + " :: " + str(status[ii]['map']), 
                            inline=False)
        await client.send_message(message.channel, embed=embed)
# outputs on active servers, i.e. with players    
    if message.content.startswith('!active'):
        
        status = {}
        
        with valve.source.master_server.MasterServerQuerier() as msq:
            try:
                for address in msq.find(gamedir="NeotokyoSource"):
                    with valve.source.a2s.ServerQuerier(address) as server:
                        info = server.info()
                        players = server.players()
                        status[str(address[0]) + str(":") + str(address[1])] = dict(info)
            except valve.source.NoResponseError:
                pass

        embed = discord.Embed(description="```css\nActive Servers```", 
                              color=0x00FF00)
        if {k:v for k,v in status.items() if v['player_count'] > 0} != {}:
            for ii in {k:v for k,v in status.items() if v['player_count'] > 0}.keys():
                    embed.add_field(name=status[ii]['server_name'], 
                            value=str(status[ii]['player_count']) + "/" + str(status[ii]['max_players']) + " :: " + str(status[ii]['map']), 
                            inline=False)
        else:
            embed.add_field(name="No one is playing",
                            value=":(",
                            inline=False)
            
        await client.send_message(message.channel, embed=embed)

@client.event
async def on_ready():
    print('Logged in as')
    print(client.user.name)
    print(client.user.id)
    print('------')

client.run("TOKEN")

RuntimeError: Cannot close a running event loop

Logged in as
Taiko
453068835336224768
------


In [69]:
embed = discord.Embed(title="Tile", description="Desc", color=0x00ff00)
embed.add_field(name="Fiel1", value="hi", inline=False)
embed.add_field(name="Field2", value="hi2", inline=False)
print(embed)
print(embed.to_dict())
test = embed.to_dict()
embeds = test

{'fields': [{'inline': False, 'name': 'Fiel1', 'value': 'hi'}, {'inline': False, 'name': 'Field2', 'value': 'hi2'}], 'color': 65280, 'type': 'rich', 'description': 'Desc', 'title': 'Tile'}


In [82]:
discord.to_Embed(discord.Embed(title="Tile", description="Desc", color=0x00ff00).to_dict())

AttributeError: module 'discord' has no attribute 'to_Embed'

In [105]:
embed = discord.Embed(title="title ~~(did you know you can have markdown here too?)~~", 
                      description="this supports [named links](https://discordapp.com) on top of the previously shown subset of markdown. ```\nyes, even code blocks```", 
                      color=0x00ff00, 
                      url="https://discordapp.com", 
                      footer=(icon_url="https://cdn.discordapp.com/embed/avatars/0.png", 
                              text="footer text"))

SyntaxError: invalid syntax (<ipython-input-105-3aba52361a17>, line 5)

In [121]:
message.channel.send({embed: {
  color: 3447003,
  description: "A very simple Embed!"
}})

NameError: name 'message' is not defined

In [85]:
t2 = {
  "content": "this `supports` __a__ **subset** *of* ~~markdown~~ 😃 ```js\nfunction foo(bar) {\n  console.log(bar);\n}\n\nfoo(1);```",
  "embed": {
    "title": "title ~~(did you know you can have markdown here too?)~~",
    "description": "this supports [named links](https://discordapp.com) on top of the previously shown subset of markdown. ```\nyes, even code blocks```",
    "url": "https://discordapp.com",
    "color": 9626272,
    "timestamp": "2018-06-04T10:01:58.009Z",
    "footer": {
      "icon_url": "https://cdn.discordapp.com/embed/avatars/0.png",
      "text": "footer text"
    },
    "thumbnail": {
      "url": "https://cdn.discordapp.com/embed/avatars/0.png"
    },
    "image": {
      "url": "https://cdn.discordapp.com/embed/avatars/0.png"
    },
    "author": {
      "name": "author name",
      "url": "https://discordapp.com",
      "icon_url": "https://cdn.discordapp.com/embed/avatars/0.png"
    },
    "fields": [
      {
        "name": "🤔",
        "value": "some of these properties have certain limits..."
      },
      {
        "name": "😱",
        "value": "try exceeding some of them!"
      },
      {
        "name": "🙄",
        "value": "an informative error should show up, and this view will remain as-is until all issues are fixed"
      },
      {
        "name": "<:thonkang:219069250692841473>",
        "value": "these last two",
        "inline": True
      },
      {
        "name": "<:thonkang:219069250692841473>",
        "value": "are inline fields",
        "inline": True
      }
    ]
  }
}

In [92]:
Class(t2)

NameError: name 'Class' is not defined

In [10]:
await client.send_message(message.channel, msg, { embed = embed })

SyntaxError: invalid syntax (<ipython-input-10-fa3f8351af2d>, line 1)

In [23]:
em = discord.Embed(title='My Embed Title', description='My Embed Content.', colour=0xDEADBF)
em.set_author(name='Someone', icon_url=client.user.default_avatar_url)
json(em)

NameError: name 'json' is not defined

In [28]:
import json

In [34]:
type(em)

discord.embeds.Embed

In [42]:
Embed(em).

SyntaxError: invalid syntax (<ipython-input-42-3be87c22e2b7>, line 1)

In [49]:
# server.name = '1'
msg = "**Name:** " + '1' + "\n"
msg += "**Total members:** " + '1' + "\n"
msg += "**Server owner:** " + '2' + "\n"
msg += "**Server region:** " + '3' + "\n"
msg += "**Created at:** " + '9'

em = discord.Embed(description=msg)

print(msg)

**Name:** 1
**Total members:** 1
**Server owner:** 2
**Server region:** 3
**Created at:** 9


In [51]:
em.from_dict()

AttributeError: 'Embed' object has no attribute 'from_dict'